In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import xlnet
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import model_utils

In [3]:
import sentencepiece as spm
from prepro_utils import preprocess_text, encode_ids

sp_model = spm.SentencePieceProcessor()
sp_model.Load('alxlnet-base/sp10m.cased.v9.model')

True

In [4]:
from rules import normalized_chars
import random

laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'hoho',
    'hewhew',
    'uwu',
    'sksk',
    'ksks',
    'gituu',
    'gitu',
    'mmeeooww',
    'meow',
    'alhamdulillah',
    'muah',
    'mmuahh',
    'hehe',
    'salamramadhan',
    'happywomensday',
    'jahagaha',
    'ahakss',
    'ahksk'
}

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def cleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    
    chars = '.,/'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    x = []
    for word in string:
        word = word.lower()
        if any([laugh in word for laugh in laughing]):
            if random.random() >= 0.5:
                x.append(word)
        else:
            x.append(word)
    string = [w.title() if w[0].isupper() else w for w in x]
    return ' '.join(string)

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from unidecode import unidecode
import re

df = pd.read_csv('../Malaya-Dataset/news-sentiment/sentiment-data-v2.csv')
Y = LabelEncoder().fit_transform(df.label)

with open('../Malaya-Dataset/polarity/polarity-negative-translated.txt','r') as fopen:
    texts = fopen.read().split('\n')
labels = [0] * len(texts)

with open('../Malaya-Dataset/polarity/polarity-positive-translated.txt','r') as fopen:
    positive_texts = fopen.read().split('\n')
labels += [1] * len(positive_texts)
texts += positive_texts
texts += df.iloc[:,1].tolist()
labels += Y.tolist()

assert len(labels) == len(texts)

In [6]:
import json
with open('../Malaya-Dataset/multidomain-sentiment/bm-amazon.json') as fopen:
    amazon = json.load(fopen)
    
with open('../Malaya-Dataset/multidomain-sentiment/bm-imdb.json') as fopen:
    imdb = json.load(fopen)
    
with open('../Malaya-Dataset/multidomain-sentiment/bm-yelp.json') as fopen:
    yelp = json.load(fopen)
    
texts += amazon['negative']
labels += [0] * len(amazon['negative'])
texts += amazon['positive']
labels += [1] * len(amazon['positive'])

texts += imdb['negative']
labels += [0] * len(imdb['negative'])
texts += imdb['positive']
labels += [1] * len(imdb['positive'])

texts += yelp['negative']
labels += [0] * len(yelp['negative'])
texts += yelp['positive']
labels += [1] * len(yelp['positive'])

In [7]:
with open('../sentiment/strong-positives.json') as fopen:
    positives = json.load(fopen)
    positives = random.sample(positives, 500000)
    
len(positives)

500000

In [8]:
with open('../sentiment/strong-negatives.json') as fopen:
    negatives = json.load(fopen)
    negatives = random.sample(negatives, 500000)
    
len(negatives)

500000

In [9]:
texts += negatives
labels += [0] * len(negatives)
texts += positives
labels += [1] * len(positives)

In [10]:
from tqdm import tqdm

for i in tqdm(range(len(texts))):
    texts[i] = cleaning(texts[i])

100%|██████████| 1016725/1016725 [01:12<00:00, 13964.19it/s]


In [11]:
actual_t, actual_l = [], []

for i in tqdm(range(len(texts))):
    if len(texts[i]) > 2:
        actual_t.append(texts[i])
        actual_l.append(labels[i])

100%|██████████| 1016725/1016725 [00:00<00:00, 1326728.32it/s]


In [12]:
from prepro_utils import preprocess_text, encode_ids

def tokenize_fn(text):
    text = preprocess_text(text, lower= False)
    return encode_ids(sp_model, text)

In [13]:
SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

special_symbols = {
    "<unk>"  : 0,
    "<s>"    : 1,
    "</s>"   : 2,
    "<cls>"  : 3,
    "<sep>"  : 4,
    "<pad>"  : 5,
    "<mask>" : 6,
    "<eod>"  : 7,
    "<eop>"  : 8,
}

VOCAB_SIZE = 32000
UNK_ID = special_symbols["<unk>"]
CLS_ID = special_symbols["<cls>"]
SEP_ID = special_symbols["<sep>"]
MASK_ID = special_symbols["<mask>"]
EOD_ID = special_symbols["<eod>"]

def XY(left_train):
    X, segments, masks = [], [], []
    for i in tqdm(range(len(left_train))):
        tokens_a = tokenize_fn(left_train[i])
        segment_id = [SEG_ID_A] * len(tokens_a)
        tokens_a.append(SEP_ID)
        tokens_a.append(CLS_ID)
        segment_id.append(SEG_ID_A)
        segment_id.append(SEG_ID_CLS)
        input_mask = [0] * len(tokens_a)
        X.append(tokens_a)
        segments.append(segment_id)
        masks.append(input_mask)
    return X, segments, masks

In [14]:
X, segments, masks = XY(actual_t)

100%|██████████| 1016724/1016724 [02:02<00:00, 8324.53it/s]


In [16]:
kwargs = dict(
      is_training=True,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.1,
      dropatt=0.1,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='alxlnet-base/config.json')

In [17]:
epoch = 2
batch_size = 40
warmup_proportion = 0.1
num_train_steps = int(len(X) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)
print(num_train_steps, num_warmup_steps)
learning_rate = 2e-5

training_parameters = dict(
      decay_method = 'poly',
      train_steps = num_train_steps,
      learning_rate = learning_rate,
      warmup_steps = num_warmup_steps,
      min_lr_ratio = 0.0,
      weight_decay = 0.00,
      adam_epsilon = 1e-8,
      num_core_per_host = 1,
      lr_layer_decay_rate = 1,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clip = 1.0,
      clamp_len=-1,)

50836 5083


In [18]:
class Parameter:
    def __init__(self, decay_method, warmup_steps, weight_decay, adam_epsilon, 
                num_core_per_host, lr_layer_decay_rate, use_tpu, learning_rate, train_steps,
                min_lr_ratio, clip, **kwargs):
        self.decay_method = decay_method
        self.warmup_steps = warmup_steps
        self.weight_decay = weight_decay
        self.adam_epsilon = adam_epsilon
        self.num_core_per_host = num_core_per_host
        self.lr_layer_decay_rate = lr_layer_decay_rate
        self.use_tpu = use_tpu
        self.learning_rate = learning_rate
        self.train_steps = train_steps
        self.min_lr_ratio = min_lr_ratio
        self.clip = clip
        
training_parameters = Parameter(**training_parameters)

In [19]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.float32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        xlnet_model = xlnet.XLNetModel(
            xlnet_config=xlnet_config,
            run_config=xlnet_parameters,
            input_ids=tf.transpose(self.X, [1, 0]),
            seg_ids=tf.transpose(self.segment_ids, [1, 0]),
            input_mask=tf.transpose(self.input_masks, [1, 0]))
        
        output_layer = xlnet_model.get_sequence_output()
        output_layer = tf.transpose(output_layer, [1, 0, 2])
        
        self.logits_seq = tf.layers.dense(output_layer, dimension_output)
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer, self.learning_rate, _ = model_utils.get_train_op(training_parameters, self.cost)
        
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [20]:
dimension_output = 2

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())




INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [21]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [22]:
tvars = tf.trainable_variables()
checkpoint = 'alxlnet-base/model.ckpt'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [23]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from alxlnet-base/model.ckpt


In [24]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_masks, test_masks, train_segments, test_segments, train_Y, test_Y = train_test_split(
    X, segments, masks, actual_l, test_size = 0.2
)

In [25]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

i = 0
index = 4
batch_x = train_X[i : index]
batch_y = train_Y[i : index]
batch_masks = train_masks[i : index]
batch_segments = train_segments[i : index]
batch_x = pad_sequences(batch_x, padding='post')
batch_segments = pad_sequences(batch_segments, padding='post', value = 4)
batch_masks = pad_sequences(batch_masks, padding='post', value = 1)

sess.run(
    [model.accuracy, model.cost],
    feed_dict = {
        model.X: batch_x,
        model.Y: batch_y,
        model.segment_ids: batch_segments,
        model.input_masks: batch_masks,
    },
)

[0.75, 0.44652295]

In [26]:
from tqdm import tqdm
import time

for EPOCH in range(epoch):

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = train_X[i : index]
        batch_y = train_Y[i : index]
        batch_masks = train_masks[i : index]
        batch_segments = train_segments[i : index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_segments = pad_sequences(batch_segments, padding='post', value = 4)
        batch_masks = pad_sequences(batch_masks, padding='post', value = 1)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
                model.segment_ids: batch_segments,
                model.input_masks: batch_masks,
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = test_X[i : index]
        batch_y = test_Y[i : index]
        batch_masks = test_masks[i : index]
        batch_segments = test_segments[i : index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_segments = pad_sequences(batch_segments, padding='post', value = 4)
        batch_masks = pad_sequences(batch_masks, padding='post', value = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
                model.segment_ids: batch_segments,
                model.input_masks: batch_masks,
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
        
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/20335 [00:00<?, ?it/s]

epoch: 0, training loss: 0.125214, training acc: 0.952616, valid loss: 0.040826, valid acc: 0.987874



test minibatch loop: 100%|██████████| 5084/5084 [08:46<00:00,  9.66it/s, accuracy=1, cost=0.00406]   

epoch: 1, training loss: 0.029429, training acc: 0.991042, valid loss: 0.029999, valid acc: 0.991227



In [27]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'alxlnet-base-sentiment/model.ckpt')

'alxlnet-base-sentiment/model.ckpt'

In [28]:
kwargs = dict(
      is_training=False,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='alxlnet-base/config.json')

In [29]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())

INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>


/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [30]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'alxlnet-base-sentiment/model.ckpt')

INFO:tensorflow:Restoring parameters from alxlnet-base-sentiment/model.ckpt


In [31]:
real_Y, predict_Y = [], []

pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = test_X[i : index]
    batch_y = test_Y[i : index]
    batch_masks = test_masks[i : index]
    batch_segments = test_segments[i : index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_segments = pad_sequences(batch_segments, padding='post', value = 4)
    batch_masks = pad_sequences(batch_masks, padding='post', value = 1)
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segments,
                model.input_masks: batch_masks
            },
    ), 1, ).tolist()
    real_Y += batch_y

test minibatch loop: 100%|██████████| 5084/5084 [07:27<00:00, 11.36it/s]


In [32]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['negative', 'positive'],digits=5
    )
)

              precision    recall  f1-score   support

    negative    0.99286   0.99090   0.99188    101710
    positive    0.99091   0.99287   0.99189    101635

    accuracy                        0.99188    203345
   macro avg    0.99188   0.99188   0.99188    203345
weighted avg    0.99188   0.99188   0.99188    203345



In [33]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'Placeholder_3',
 'model/transformer/r_w_bias',
 'model/transformer/r_r_bias',
 'model/transformer/word_embedding/lookup_table',
 'model/transformer/word_embedding/lookup_table_2',
 'model/transformer/r_s_bias',
 'model/transformer/seg_embed',
 'model/transformer/layer_shared/rel_attn/q/kernel',
 'model/transformer/layer_shared/rel_attn/k/kernel',
 'model/transformer/layer_shared/rel_attn/v/kernel',
 'model/transformer/layer_shared/rel_attn/r/kernel',
 'model/transformer/layer_shared/rel_attn/o/kernel',
 'model/transformer/layer_shared/rel_attn/LayerNorm/gamma',
 'model/transformer/layer_shared/ff/layer_1/kernel',
 'model/transformer/layer_shared/ff/layer_1/bias',
 'model/transformer/layer_shared/ff/layer_2/kernel',
 'model/transformer/layer_shared/ff/layer_2/bias',
 'model/transformer/layer_shared/ff/LayerNorm/gamma',
 'dense/kernel',
 'dense/bias',
 'logits_seq',
 'logits']

In [34]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [35]:
freeze_graph('alxlnet-base-sentiment', strings)

INFO:tensorflow:Restoring parameters from alxlnet-base-sentiment/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 21 variables.
INFO:tensorflow:Converted 21 variables to const ops.
7390 ops in the final graph.


In [36]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'alxlnet-base-sentiment/frozen_model.pb'
outPutname = "v34/sentiment/alxlnet-base-sentiment.pb"

s3 = boto3.client('s3')

s3.upload_file(Key,bucketName,outPutname)